## Import

In [1]:
import wandb

In [2]:
wandb.login()

wandb: Currently logged in as: devhunmin (use `wandb login --relogin` to force relogin)


True

In [3]:
wandb.init(project="efficientdet",
           config={
               "model":'efficientdet-b4'
           })

In [1]:
from pycocotools.coco import COCO
import gluoncv.utils as gcv
from torch.utils.data.sampler import WeightedRandomSampler
import torch
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import warnings
from torch.optim.lr_scheduler import _LRScheduler
import math
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [4]:
torch.__version__

'1.6.0'

In [7]:
!nvidia-smi

Tue May 18 00:33:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   41C    P0    44W / 250W |  15612MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [7]:
batch_size = 4
learning_rate = 1e-4
num_epochs = 200
seed = 41

In [8]:
seed_everything(seed)

In [9]:
IMAGENET_DEFAULT_MEAN = [x * 255 for x in (0.485, 0.456, 0.406)]
IMAGENET_DEFAULT_STD = [x * 255 for x in (0.229, 0.224, 0.225)]

class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''
    def __init__(self, annotation, data_dir, test_mode, transforms):
        super().__init__()
        self.img_size=256
        self.mosaic_border = [512//2, 512//2]
        self.test_mode = test_mode
        self.obj_label = []
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms
        
        if self.test_mode==False:
            for index in tqdm(range(len(self.coco.getImgIds()))):
                image_id = self.coco.getImgIds(imgIds=index)
                image_info = self.coco.loadImgs(image_id)[0]
                ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
                anns = self.coco.loadAnns(ann_ids)
                obj_list = np.array([x['category_id'] for x in anns])
                self.get_obj_label(obj_list)
    
    #   0          1             2        3        4       5     6         7            8          9         10
    # UNKNOWN, General trash, Paper, Paper pack, Metal, Glass, Plastic, Styrofoam, Plastic bag, Battery, Clothing
    #  160,            2782,    9311,     659,    562   , 610 ,  3090,     1343    ,  7643,       63,        177
    def get_obj_label(self,obj_list):
        if 9.0 in list(np.unique(obj_list)): self.obj_label.append(0)
        elif 0.0 in list(np.unique(obj_list)): self.obj_label.append(1)
        elif 10.0 in list(np.unique(obj_list)): self.obj_label.append(2)
        elif 4.0 in list(np.unique(obj_list)): self.obj_label.append(3)
        elif 5.0 in list(np.unique(obj_list)): self.obj_label.append(4)
        elif 3.0 in list(np.unique(obj_list)): self.obj_label.append(5)
        elif 7.0 in list(np.unique(obj_list)): self.obj_label.append(6)
        elif 1.0 in list(np.unique(obj_list)): self.obj_label.append(7)
        elif 6.0 in list(np.unique(obj_list)): self.obj_label.append(8)
        elif 8.0 in list(np.unique(obj_list)): self.obj_label.append(9)
        else: self.obj_label.append(10)

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image, boxes, labels = self.load_image_and_boxes(index)

        if self.test_mode:
            image, boxes, labels = self.load_image_and_boxes(index)
        else:
            p = random.random()
            if p > 0.6:
                image, boxes, labels = self.load_mixup_image_and_boxes(index) # p=0.4
            elif p > 0.3:
                image, boxes, labels = self.load_mosaic(index) # p=0.3
            else:
                image, boxes, labels = self.load_image_and_boxes(index) # p=0.3

            
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        
        # Multi Scale Training 의 실마리
        target['img_size'] = torch.tensor([(512, 512)])
        target['img_scale'] = torch.tensor([1.])
        
        if self.transforms:
            for i in range(10):
                sample = {
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': target['labels']
                }
                sample = self.transforms(**sample)
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    target['labels'] = torch.tensor(sample['labels'])
                    break
        
        return image, target, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())
    
    def load_image_and_boxes(self, index):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = (image - IMAGENET_DEFAULT_MEAN) / IMAGENET_DEFAULT_STD

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        boxes = np.array([x['bbox'] for x in anns])
        
        labels = np.array([x['category_id'] for x in anns])
        #labels = torch.as_tensor(labels, dtype=torch.int64)

        # boxex (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        return image, boxes, labels
    
    def load_mixup_image_and_boxes(self, index):
        image, boxes, labels = self.load_image_and_boxes(index)
        r_image, r_boxes, r_labels = self.load_image_and_boxes(random.randint(0, len(self.coco.getImgIds()) - 1))
        lam = np.clip(np.random.beta(1.0, 1.0), 0.35, 0.65)
        image = lam * image + (1 - lam) * r_image
        return image, np.vstack((boxes, r_boxes)).astype(np.int32), np.concatenate((labels, r_labels))
    
    def load_mosaic(self, index):
        # Modify -> Normal 1/4 image
        boxes4 = []
        labels4 = []
        s = self.img_size
        h = s
        w = s
        #xc, yc = [int(random.uniform(-x, 2*s+x)) for x in self.mosaic_border]
        xc, yc = self.mosaic_border
        indices = [index] + [random.randint(0, len(self.coco.getImgIds()) - 1) for _ in range(3)]
        for i, index in enumerate(indices):
            image, y, x = self.load_image_and_boxes(index)

            boxes = y.copy()

            if i==0: # top left
                img4 = np.full((s*2,s*2, image.shape[2]), 114/255.0, dtype=np.float32)
                x1a, y1a, x2a, y2a = max(xc-w,0), max(yc-h,0), xc, yc
                x1b, y1b, x2b, y2b = w - (x2a-x1a), h-(y2a-y1a), w, h
            elif i==1: # top right
                x1a, y1a, x2a, y2a = xc, max(yc-h,0), min(xc+w, s*2), yc
                x1b, y1b, x2b, y2b = 0, h - (y2a-y1a), min(w, x2a-x1a), h
            elif i==2: # bottom left
                x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s*2, yc+h)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a-y1a, h)
            elif i==3: # bottom right
                x1a, y1a, x2a, y2a = xc, yc, min(xc+w, s*2), min(s*2,yc+h)
                x1b, y1b, x2b, y2b = 0, 0, min(w,x2a-x1a), min(y2a-y1a,h)

            img4[y1a:y2a, x1a:x2a]=image[y1b:y2b, x1b:x2b]
            padw = x1a - x1b
            padh = y1a - y1b

            labels = x.copy()
            boxes = y.copy()

            if len(labels)>0:
                boxes[:,0]=(y[:,0])+padw
                boxes[:,1]=(y[:,1])+padh
                boxes[:,2]=(y[:,2])+padw
                boxes[:,3]=(y[:,3])+padh
          
            boxes4.append(boxes)
            labels4.append(labels)
      
        labels4 = np.concatenate(labels4,0)
        boxes4 = np.concatenate(boxes4,0)
        np.clip(boxes4, 0, 2*s, out=boxes4)
        indicies = []
        for i, box in enumerate(boxes4):
            if box[0] >= box[2] or box[1] >= box[3]:
                indicies.append(i)

        boxes4 = np.delete(boxes4,indicies, axis=0)
        labels4 = np.delete(labels4, indicies, axis=0)

        return img4, boxes4, labels4

In [10]:
def collate_fn(batch):
    return tuple(zip(*batch))

def get_train_transform():
    return A.Compose([
        A.OneOf([
            A.RandomCrop(384,384,p=1),
            A.RandomCrop(256,256,p=1),
        ],p=0.3),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.3),
        A.VerticalFlip(p=0.4),
        A.Resize(256,256),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels'])
    )

def get_valid_transform():
    return A.Compose([
        A.Resize(256,256),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels'])
    )

In [11]:
train_annotation = '../input/data/train.json'
val_annotation = '../input/data/val.json'
data_dir = '../input/data'
train_dataset = CustomDataset(train_annotation, data_dir, False, get_train_transform())
val_dataset = CustomDataset(val_annotation, data_dir, True, get_valid_transform())


# weighted sampler
counts = np.bincount(train_dataset.obj_label)
labels_weights = 1. / counts
weights = labels_weights[train_dataset.obj_label]
weight_sampler=WeightedRandomSampler(weights, len(weights))

train_data_loader=DataLoader(train_dataset,batch_size=batch_size,sampler=weight_sampler,collate_fn=collate_fn,num_workers=0)

#train_data_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn,num_workers=0)
val_data_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False,collate_fn=collate_fn,num_workers=0)

loading annotations into memory...


 36%|███▌      | 947/2617 [00:00<00:00, 9466.04it/s]

Done (t=4.06s)
creating index...
index created!


100%|██████████| 2617/2617 [00:00<00:00, 8868.53it/s]


loading annotations into memory...
Done (t=1.41s)
creating index...
index created!


In [12]:
def get_model():
    #tf_efficientdet_d5_ap
    config = get_efficientdet_config('tf_efficientdet_d4')
    config.image_size = (256,256)
    config.norm_kwargs=dict(eps=.001, momentum=.01)
    net = EfficientDet(config, pretrained_backbone=True)
    checkpoint = torch.load('./pretrained_weight/tf_efficientdet_d4_49-f56376d9.pth')
    net.load_state_dict(checkpoint)

    net.reset_head(num_classes=11)
    net.class_net = HeadNet(config, num_outputs=11)
    return DetBenchTrain(net, config)

In [13]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [14]:
class EarlyStopping:
    def __init__(self, patience=100, verbose=False, delta=0, path=None):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.path = path

    def __call__(self, val_mAP, model):
        score = val_mAP
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        model.eval()
        torch.save(model.state_dict(), self.path)

In [15]:
def freeze_backbone(m):
    classname = m.__class__.__name__
    for ntl in ['EfficientNet', 'BiFPN']:
        if ntl in classname:
            for param in m.parameters():
                param.requires_grad = False

def Unfreeze_backbone(m):
    classname = m.__class__.__name__
    for ntl in ['EfficientNet', 'BiFPN']:
        if ntl in classname:
            for param in m.parameters():
                param.requires_grad = True

In [16]:
def train_fn(model, train_data_loader, val_data_loader, optimizer, scheduler, device):
    model.apply(freeze_backbone)
    model.train()
    train_loss = AverageMeter()
    train_cls_loss = AverageMeter()
    train_box_loss = AverageMeter()
    save_path = './saved/checkpoint.pth'
    early_stop = EarlyStopping(path=save_path)
    for epoch in range(num_epochs):
        train_loss.reset()
        train_cls_loss.reset()
        train_box_loss.reset()
        model.train()
        if epoch == 30:
            model.apply(Unfreeze_backbone)
        for images, targets, image_ids in tqdm(train_data_loader):
            # gpu 계산을 위해 image.to(device)
            images = torch.stack(images)
            images = images.to(device).float()
            current_batch_size = images.shape[0]

            targets2 = {}
            targets2['bbox'] = [target['boxes'].to(device).float() for target in targets] # variable number of instances, so the entire structure can be forced to tensor
            targets2['cls'] = [target['labels'].to(device).float() for target in targets]
            targets2['image_id'] = torch.tensor([target['image_id'] for target in targets]).to(device).float()
            targets2['img_scale'] = torch.tensor([target['img_scale'] for target in targets]).to(device).float()
            targets2['img_size'] = torch.tensor([(256, 256) for target in targets]).to(device).float()

            # calculate loss
            losses, cls_loss, box_loss = model(images, targets2).values()

            train_loss.update(losses.detach().item(), current_batch_size)
            train_cls_loss.update(cls_loss.detach().item(), current_batch_size)
            train_box_loss.update(box_loss.detach().item(), current_batch_size)

            # backward
            optimizer.zero_grad()
            losses.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 10.0)
            optimizer.step()
        
        # Evaluation
        val_loss, val_mAP = vali_fn(val_data_loader,model,device)
        # Scheduler Step
        scheduler.step(val_mAP)
        
        print(f"\nEpoch #{epoch+1} train loss: [{train_loss.avg}] train cls loss : [{train_cls_loss.avg}] train box loss : [{train_box_loss.avg}] validation mAP : [{val_mAP}] \n")
        wandb.log({"Train Loss":train_loss.avg, "Validation mAP@50":val_mAP})
        early_stop(val_mAP,model)
        if early_stop.early_stop:
            print('Stop Training.....')
            break

In [17]:
def vali_fn(val_data_loader, model, device):
    model.eval()
    vali_loss = AverageMeter()
    vali_mAP = AverageMeter()
    metric = gcv.metrics.VOCMApMetric(iou_thresh=0.5)
    with torch.no_grad():
        for images, targets, image_ids in tqdm(val_data_loader):
            # gpu 계산을 위해 image.to(device)
            images = torch.stack(images)
            images = images.to(device).float()

            current_batch_size = images.shape[0]

            targets2 = {}
            targets2['bbox'] = [target['boxes'].to(device).float() for target in targets] # variable number of instances, so the entire structure can be forced to tensor
            targets2['cls'] = [target['labels'].to(device).float() for target in targets]
            targets2['image_id'] = torch.tensor([target['image_id'] for target in targets]).to(device).float()
            targets2['img_scale'] = torch.tensor([target['img_scale'] for target in targets]).to(device).float()
            targets2['img_size'] = torch.tensor([(256, 256) for target in targets]).to(device).float()

            outputs = model(images, targets2)

            loss = outputs['loss']
            det = outputs['detections']

            # Calc Metric
            for i in range(0, len(det)):
                pred_scores=det[i, :, 4].cpu().unsqueeze_(0).numpy()
                condition=(pred_scores > 0)[0]
                gt_boxes=targets2['bbox'][i].cpu().unsqueeze_(0).numpy()[...,[1,0,3,2]] #move to PASCAL VOC from yxyx format
                gt_labels=targets2['cls'][i].cpu().unsqueeze_(0).numpy()

                pred_bboxes=det[i, :, 0:4].cpu().unsqueeze_(0).numpy()[:, condition, :]
                pred_labels=det[i, :, 5].cpu().unsqueeze_(0).numpy()[:, condition]
                pred_scores=pred_scores[:, condition]
                metric.update(
                  pred_bboxes=pred_bboxes,
                  pred_labels=pred_labels,
                  pred_scores=pred_scores,
                  gt_bboxes=gt_boxes,
                  gt_labels=gt_labels)

            vali_mAP.update(metric.get()[1], current_batch_size)
            vali_loss.update(loss.detach().item(), current_batch_size)
    
    # validation loss
    return vali_loss.avg, vali_mAP.avg

In [18]:
# Model
model = get_model()
model.to(device)
model.eval()
# setting
#optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
params = [p for p in model.parameters() if p.requires_grad]
#optimizer = torch.optim.SGD(params, lr=learning_rate, momentum=0.9, nesterov=True)
optimizer = torch.optim.AdamW(params, lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.7, patience=3,threshold_mode='abs',min_lr=1e-7, verbose=True)
wandb.watch(model)

In [ ]:
train_fn(model, train_data_loader, val_data_loader, optimizer, scheduler, device)

100%|██████████| 41/41 [00:26<00:00,  1.52it/s]



Epoch #1 train loss: [33.58281755661481] train cls loss : [33.11349672186133] train box loss : [0.00938641691183971] validation mAP : [0.0013665021276880998] 



100%|██████████| 41/41 [00:28<00:00,  1.42it/s]



Epoch #2 train loss: [1.5335163019702869] train cls loss : [1.1191812148292783] train box loss : [0.008286701644855144] validation mAP : [0.011167019679788698] 



100%|██████████| 41/41 [00:28<00:00,  1.45it/s]


...

  0%|          | 0/164 [00:00<?, ?it/s]

Epoch   151: reducing learning rate of group 0 to 1.0000e-07.

Epoch #151 train loss: [0.6805663094249079] train cls loss : [0.40734075062776914] train box loss : [0.005464511214234831] validation mAP : [0.3638339615717992] 



  0%|          | 0/164 [00:00<?, ?it/s]


Epoch #152 train loss: [0.6738284998464821] train cls loss : [0.4039558396568998] train box loss : [0.005397453143826857] validation mAP : [0.37200186490583476] 



 43%|████▎     | 71/164 [01:26<01:49,  1.18s/it]